In [211]:
# Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
import matplotlib as mpl
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

In [267]:
# Loading data
notas_treino = pd.read_csv('train.csv')

# Verifying and removing columns with unique values
counts = notas_treino.nunique()
to_del = [i for i,v in enumerate(counts) if v == 1]

# Columns to be removed
to_del1 = [1,2,3,4,5,7,8,9,10,11,12,13,14,15,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,68,69,70,71,72,77,78,79,80,81,82,83,84,85,86]
df_n_t = notas_treino.drop(notas_treino.columns[to_del], axis=1)
df_n_t = df_n_t.drop(df_n_t.columns[to_del1], axis=1)
df_notas_treino = df_n_t.iloc[:,1:-50]

# Trasforming TP_SEXO in numeric
sexo = {"TP_SEXO": {"M": 1, "F": 2}}
df_notas_treino.replace(sexo, inplace=True)

In [268]:
# Data types
df_notas_treino.dtypes

NU_IDADE             int64
TP_ANO_CONCLUIU      int64
TP_ESCOLA            int64
TP_PRESENCA_CN       int64
TP_PRESENCA_CH       int64
TP_PRESENCA_LC       int64
NU_NOTA_CN         float64
NU_NOTA_CH         float64
NU_NOTA_LC         float64
NU_NOTA_MT         float64
NU_NOTA_COMP1      float64
NU_NOTA_COMP2      float64
NU_NOTA_COMP3      float64
NU_NOTA_COMP4      float64
NU_NOTA_COMP5      float64
NU_NOTA_REDACAO    float64
dtype: object

In [269]:
# Replacing the values equal to zero with the average of the data
df_notas_treino = df_notas_treino.fillna(df_notas_treino.mean()['NU_NOTA_COMP1':'NU_NOTA_REDACAO'])

df_notas_treino['NU_NOTA_CN'] = df_notas_treino['NU_NOTA_CN'].fillna(0)
df_notas_treino['NU_NOTA_CH'] = df_notas_treino['NU_NOTA_CH'].fillna(0)
df_notas_treino['NU_NOTA_LC'] = df_notas_treino['NU_NOTA_LC'].fillna(0)
df_notas_treino['NU_NOTA_MT'] = df_notas_treino['NU_NOTA_MT'].fillna(0)

#Truncation and rounding
df_notas_treino['NU_NOTA_REDACAO'] = df_notas_treino['NU_NOTA_REDACAO'].round(decimals = 0)
df_notas_treino['NU_NOTA_COMP1'] = df_notas_treino['NU_NOTA_COMP1'].round(decimals = 0)
df_notas_treino['NU_NOTA_COMP2'] = df_notas_treino['NU_NOTA_COMP2'].round(decimals = 0)
df_notas_treino['NU_NOTA_COMP3'] = df_notas_treino['NU_NOTA_COMP3'].round(decimals = 0)
df_notas_treino['NU_NOTA_COMP4'] = df_notas_treino['NU_NOTA_COMP4'].round(decimals = 0)
df_notas_treino['NU_NOTA_COMP5'] = df_notas_treino['NU_NOTA_COMP5'].round(decimals = 0)

In [270]:
# Verifying missing values
print(df_notas_treino.isna().sum())

NU_IDADE           0
TP_ANO_CONCLUIU    0
TP_ESCOLA          0
TP_PRESENCA_CN     0
TP_PRESENCA_CH     0
TP_PRESENCA_LC     0
NU_NOTA_CN         0
NU_NOTA_CH         0
NU_NOTA_LC         0
NU_NOTA_MT         0
NU_NOTA_COMP1      0
NU_NOTA_COMP2      0
NU_NOTA_COMP3      0
NU_NOTA_COMP4      0
NU_NOTA_COMP5      0
NU_NOTA_REDACAO    0
dtype: int64


In [271]:
df_notas_treino.shape

(13730, 16)

In [273]:
# Data evaluation report
profile = ProfileReport(df_notas_treino)
profile

In [ ]:
# Moving target column to the end
cols_to_move = ['NU_NOTA_MT']
new_cols = np.hstack((df_notas_treino.columns.difference(cols_to_move), cols_to_move))
df_notas_treino = df_notas_treino.loc[:, new_cols]

In [ ]:
# Normalizing data
df_norm = (df_notas_treino - df_notas_treino.mean()) / (df_notas_treino.max() - df_notas_treino.min())

# Collecting x and y
X = df_norm.iloc[:, 0:15]  
y = df_notas_treino.iloc[:, 15]

In [ ]:
# Trainning and test division
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [ ]:
#Transforming data in array
DT_X_train = np.array(X_train)
DT_y_train = np.array(y_train)
DT_X_test = np.array(X_test)
DT_y_test = np.array(y_test)

## Trainning the model: LinearRegression

In [231]:
# Creating the model
modelo = LinearRegression(normalize = False, fit_intercept = True)

In [232]:
# Trainning the model
modelo_v1 = modelo.fit(X_train, y_train)

In [233]:
# R2 metric of the model
r2_score(y_test, modelo_v1.fit(X_train, y_train).predict(X_test))

0.9115462637903832

## Trainning the model: Decision Tree (using DecisionTreeRegressor)

In [283]:
modelo_v2 = DecisionTreeRegressor(max_depth = None, 
                                   max_features = None, 
                                   min_samples_leaf = 7, 
                                   min_samples_split = 2)

In [281]:
modelo_v2.fit(DT_X_train, DT_y_train)

DecisionTreeRegressor(min_samples_leaf=6)

In [282]:
r2_score(DT_y_test, modelo_v2.fit(DT_X_train, DT_y_train).predict(DT_X_test))

0.8939222520393747

### Applying AdaBoostRegressor

In [284]:
modelo_v2a = AdaBoostRegressor(DecisionTreeRegressor(max_depth=None),
                          n_estimators=650)

In [285]:
modelo_v2a.fit(DT_X_train, DT_y_train)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(), n_estimators=650)

## Trainning the model: Decision Tree (using GradientBoostingRegressor)

In [344]:
# Creates the regressor
modelo_v3 = GradientBoostingRegressor(learning_rate = 0.07, max_depth = 2, random_state = 0, loss = 'ls')
# Trainning the regressor
modelo_v3.fit(DT_X_train, DT_y_train)

GradientBoostingRegressor(learning_rate=0.07, max_depth=2, random_state=0)

In [345]:
# Calculates the average error squared
mean_squared_error(DT_y_test, modelo_v3.predict(DT_X_test))

4037.7520328071323

### Preparing Test Data

In [263]:
notas_teste = pd.read_csv('test.csv')
to_del_teste = [1,2,4,5,6,7,10,11,12,13,14,15,16,17,18,19,20,24,25,26,27,31,32,39,40,41,42,43,44,45,46]
df_notas_teste = notas_teste.drop(notas_teste.columns[to_del_teste], axis=1)

# Trasforming TP_SEXO in numeric
sexo_teste = {"TP_SEXO": {"M": 1, "F": 2}}
df_notas_teste.replace(sexo_teste, inplace=True)

In [264]:
# Replacing the values equal to zero with the average of the data
df_notas_teste = df_notas_teste.fillna(df_notas_teste.mean()['NU_NOTA_COMP1':'NU_NOTA_REDACAO'])

df_notas_teste['NU_NOTA_CN'] = df_notas_teste['NU_NOTA_CN'].fillna(0)
df_notas_teste['NU_NOTA_CH'] = df_notas_teste['NU_NOTA_CH'].fillna(0)
df_notas_teste['NU_NOTA_LC'] = df_notas_teste['NU_NOTA_LC'].fillna(0)

#Truncation and rounding
df_notas_teste['NU_NOTA_REDACAO'] = df_notas_teste['NU_NOTA_REDACAO'].round(decimals = 0)
df_notas_teste['NU_NOTA_COMP1'] = df_notas_teste['NU_NOTA_COMP1'].round(decimals = 0)
df_notas_teste['NU_NOTA_COMP2'] = df_notas_teste['NU_NOTA_COMP2'].round(decimals = 0)
df_notas_teste['NU_NOTA_COMP3'] = df_notas_teste['NU_NOTA_COMP3'].round(decimals = 0)
df_notas_teste['NU_NOTA_COMP4'] = df_notas_teste['NU_NOTA_COMP4'].round(decimals = 0)
df_notas_teste['NU_NOTA_COMP5'] = df_notas_teste['NU_NOTA_COMP5'].round(decimals = 0)

In [265]:
print(df_notas_teste.isna().sum())

NU_INSCRICAO       0
NU_IDADE           0
TP_ANO_CONCLUIU    0
TP_ESCOLA          0
TP_PRESENCA_CN     0
TP_PRESENCA_CH     0
TP_PRESENCA_LC     0
NU_NOTA_CN         0
NU_NOTA_CH         0
NU_NOTA_LC         0
NU_NOTA_COMP1      0
NU_NOTA_COMP2      0
NU_NOTA_COMP3      0
NU_NOTA_COMP4      0
NU_NOTA_COMP5      0
NU_NOTA_REDACAO    0
dtype: int64


In [266]:
# Normalizing data
df_notas_teste1 = df_notas_teste.iloc[:,1:]
df_norm_teste1 = (df_notas_teste1 - df_notas_teste1.mean()) / (df_notas_teste1.max() - df_notas_teste1.min())
DT_df_norm_teste1 = np.array(df_norm_teste1)

### Making Predictions with the Trained Model

In [239]:
# Predictions with LinearRegression (modelo_v1)
predict_teste_v1 = modelo_v1.predict(df_norm_teste1)
predict_teste_v1 = predict_teste_v1.round(decimals = 1)
predict_teste_v1

array([436.2, 432.1, 336.9, ..., 532.1, 434.4,  67.1])

In [286]:
# Predictions with AdaBoostRegressor (modelo_v2a)
predict_teste_v2 = modelo_v2a.predict(DT_df_norm_teste1)
predict_teste_v2 = predict_teste_v2.round(decimals = 1)
predict_teste_v2

array([525.3, 487.1, 485.7, ..., 511.8, 457.6,   0. ])

In [346]:
# Predictions with GradientBoostingRegressor (modelo_v3) - Best result to this problem
predict_teste_v3 = modelo_v3.predict(DT_df_norm_teste1)
predict_teste_v3 = predict_teste_v3.round(decimals = 1)
predict_teste_v3

array([479.1, 512.4, 637.9, ..., 505.3, 456.4,  23.1])

### Creating file .CSV to the results

In [347]:
answer = notas_teste
answer = answer.iloc[:,0:1]
answer.insert(1, "NU_NOTA_MT", predict_teste_v3, True) 
answer.to_csv('answer.csv')